# Creating a Test System 

The data of three bus system from this website:

https://www.al-roomi.org/power-flow/3-bus-systems/system-iii

In [1]:
import pandapower as pp
import pandapower.plotting as plot
from numpy import pi
import os
import matplotlib.pyplot as plt
from pandas import DataFrame

Building the system, buses and lines  (now no transformers)

In [2]:
net = pp.create_empty_network(f_hz=60)
# net.skva=100000

In [3]:
b1 = pp.create_bus(net, vn_kv=220, name="bus1", type="b")
b2 = pp.create_bus(net, vn_kv=220, name="bus2", type="b")
b3 = pp.create_bus(net, vn_kv=220, name="bus3", type="b")

### Insert the parameters of lines 
Note: In the table of lines data, there is a typo in the reactance of X13 it should be 0.18pu instead of 0.018pu

In [4]:
Sb=100
Vb=220
Zb=Vb**2/Sb
R12=0.08*Zb; X12=0.24*Zb
R13=0.02*Zb; X13=0.06*Zb
R23=0.06*Zb; X23=0.18*Zb

In [5]:
L12=pp.create_line_from_parameters(net, from_bus=b1, to_bus=b2, length_km=1, 
                               name="Line 1-2", r_ohm_per_km= R12, x_ohm_per_km=X12, 
                               c_nf_per_km=1e-9, max_i_ka=1000)
L13=pp.create_line_from_parameters(net, from_bus=b1, to_bus=b3, length_km=1, 
                               name="Line 1-3", r_ohm_per_km= R13, x_ohm_per_km=X13, 
                               c_nf_per_km=1e-9, max_i_ka=1000)
L23=pp.create_line_from_parameters(net, from_bus=b2, to_bus=b3, length_km=1, 
                               name="Line 2-3", r_ohm_per_km= R23, x_ohm_per_km=X23, 
                               c_nf_per_km=1e-9, max_i_ka=1000) 

In [6]:
pp.create_ext_grid(net, bus=b1, vm_pu=1.05, va_degree=0, name="Grid Connection")
pp.create_gen(net, b2, p_mw=20, max_q_mvar=35, min_q_mvar=0, vm_pu=1.03, name="generator") 

0

In [7]:
pp.create_load(net, bus=b2, p_mw=50, q_mvar=20, name="Load")
pp.create_load(net, bus=b3, p_mw=60, q_mvar=25, name="Load")

1

In [8]:
net

This pandapower network includes the following parameter tables:
   - bus (3 elements)
   - load (2 elements)
   - gen (1 element)
   - ext_grid (1 element)
   - line (3 elements)

In [9]:
net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,in_service,type,power_station_trafo
0,generator,1,20.0,1.03,NaN,0.0,35.0,1.0,False,True,None,NaN


In [10]:
net.bus

,name,vn_kv,type,zone,in_service
0,bus1,220.0,b,None,True
1,bus2,220.0,b,None,True
2,bus3,220.0,b,None,True


In [11]:
net.line

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service
0,Line 1-2,None,0,1,1.0,38.72,116.16,1.000000e-09,0.0,1000.0,1.0,1,None,True
1,Line 1-3,None,0,2,1.0,9.68,29.04,1.000000e-09,0.0,1000.0,1.0,1,None,True
2,Line 2-3,None,1,2,1.0,29.04,87.12,1.000000e-09,0.0,1000.0,1.0,1,None,True


In [12]:
net.load

,name,bus,p_mw,q_mvar,const_z_percent,const_i_percent,sn_mva,scaling,in_service,type
0,Load,1,50.0,20.0,0.0,0.0,NaN,1.0,True,wye
1,Load,2,60.0,25.0,0.0,0.0,NaN,1.0,True,wye


In [37]:
# run power flow
# pp.runpp(net,enforce_q_lims=True)
pp.runpp(net,algorithm='gs', max_iteration=7, tolerance_mva=1e-03)

In [14]:
net.res_ext_grid

,p_mw,q_mvar
0,91.374287,24.068916


In [15]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.050000,0.000000,-91.374287,-24.068916
1,1.030000,-2.852064,30.000000,-5.050896
2,1.024752,-1.947046,60.000000,25.000000


In [16]:
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,22.972166,1.650784,-22.587262,-0.496073,0.384904,1.154711,0.057564,0.057564,0.057564,1.05,0.000000,1.030000,-2.852064,0.005756
1,68.402121,22.418131,-67.462181,-19.598310,0.939941,2.819822,0.179909,0.179909,0.179909,1.05,0.000000,1.024752,-1.947046,0.017991
2,-7.412906,5.546969,7.461386,-5.401531,0.048480,0.145439,0.023590,0.023590,0.023590,1.03,-2.852064,1.024752,-1.947046,0.002359


In [17]:
import numpy as np
from cmath import rect
mag=net.res_bus.vm_pu
ang=net.res_bus.va_degree

nprect = np.vectorize(rect)

c = nprect(mag, np.deg2rad(ang))
c

array([1.05      +0.j        , 1.02872418-0.05125007j,
       1.02416066-0.03481681j])

In [79]:
pp.overloaded_lines(net, max_load=100)

Int64Index([], dtype='int64')

In [86]:
pp.violated_buses(net, 0.95, 1.05)

Int64Index([], dtype='int64')

### Check out Ybus

In [127]:
Ybs=(round(net['_ppc']['internal']['Ybus'],3))
print(Ybs)

  (0, 0)	(625-1875j)
  (0, 1)	(-125+375j)
  (0, 2)	(-500+1500j)
  (1, 0)	(-125+375j)
  (1, 1)	(291.667-875j)
  (1, 2)	(-166.667+500j)
  (2, 0)	(-500+1500j)
  (2, 1)	(-166.667+500j)
  (2, 2)	(666.667-2000j)
